# Exploratory parsing

Data source: 
- Änderungsgesetz: https://dip.bundestag.de/experten-suche?term=NOT%20zusatzmerkmal:E%20AND%20vorgangstyp_notation:100&f.wahlperiode=19&f.typ=Vorgang&rows=25&sort=datum_ab
- Source law: https://www.gesetze-im-internet.de/wregg/BJNR273910017.html

## Read data

In [153]:
import pdfplumber
import regex as re

In [527]:
# open pdf
filename = '../data/0483-21.pdf'

def read_pdf_law(filename: str) -> str:
    # read all pages
    pdf_file_obj = pdfplumber.open(filename)

    # join the pages and remove newlines
    return "".join([page for page in [page.extract_text() for page in pdf_file_obj.pages] if page]).replace("\n", "")


change_law_raw = read_pdf_law(filename)

In [528]:
# extract the change requests
parsed_change_law = change_law_raw.split("wird wie folgt geändert:", 1)[1].split("Begründung", 1)[0]

print(parsed_change_law)

1.  § 2 wird wie folgt geändert: a)  Nach Absatz 2 wird folgender neuer Absatz 3 eingefügt: „(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von vergleichbarer  Schwere  wie  die  Straf-  und Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob fahrlässige Falscherklärungen 1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1 und 2 oder in vergleichbaren Registern, 2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen gesetzlichen Mindestlohn oder 3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen Arbeitsorganisation. Der für die Eintragung erforderliche Nachweis der jeweiligen schweren Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige Behörde zu treffen.“b)  Die bisherigen Abätze 3 und 4 werden 

## Structure and clean up the text

In [529]:
def structure_text(text: str) -> str:
    """Apply some structuring to the text."""
    structured_text = text
    
    # new line after number.
    res = re.findall(r"\d\.", structured_text)
    for match in set(res):
        structured_text = structured_text.replace(match, "\n" + match)
    
    # new line after § number
    #res = re.findall(r"§ \d", structured_text)
    #for match in set(res):
    #    structured_text = structured_text.replace(match, "\n" + match)
        
    # new line after letter)
    res = re.findall(r"[a-z]\)", structured_text)
    for match in set(res):
        structured_text = structured_text.replace(match, "\n" + match)
    
    # remove changes between quotation marks
    for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", structured_text, re.MULTILINE):
        structured_text = structured_text[:m.span()[0]] + structured_text[m.span()[0]:m.span()[1]].replace("\n", " ") + structured_text[m.span()[1]:]
    
    return structured_text.strip()

In [530]:
# expand text for multi-operation bullet points
def expand_text(text: str) -> str:
    outtext = ""
    pre_text = ""
    for line in text.split("\n"):
        if line[0].isdigit():
            pre_text = line
        else:
            outtext += pre_text + line + "\n"
    return outtext

In [531]:
def preprocess(raw_text: str) -> str:
    # extract the change requests
    text = raw_text.split("wird wie folgt geändert:", 1)[1].split("Begründung", 1)[0]
    # apply some structuring to the text.
    structured_text = structure_text(text)
    # expand text for multi-operation bullet points
    expanded_text = expand_text(structured_text)
    return expanded_text

In [532]:
clean_text = preprocess(change_law_raw)

In [533]:
print(clean_text)

1.  § 2 wird wie folgt geändert: a)  Nach Absatz 2 wird folgender neuer Absatz 3 eingefügt: „(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von vergleichbarer  Schwere  wie  die  Straf-  und Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere Verfehlunge n)  eingetragen,  insbesondere  vorsätzliche  oder  grob fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1 und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen Arbeitsorganisation. Der für die Eintragung erforderliche Nachweis der jeweiligen schweren Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige Behörde zu treffen.“
1.  § 2 wird wie folgt geändert: b) 

## Parse changes

In [551]:
def parse_insert_after(text: str) -> dict:
    """Take a line of text and parse it."""
    if not "eingefügt" in text:
        return None
    
    # parse the location
    location = []
    try:
        location.extend(re.search(r"§ \d{1,3}[a-z]?(\ Absatz \d)?", text).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Absatz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Satz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Nummer \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
        
    # parse text to insert
    insert_text = [text[m.span()[0]:m.span()[1]] for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", text, re.MULTILINE)]
    
    return {"location": location, "text": insert_text, "how": "insert_after"}

In [552]:
def parse_replace(text: str) -> dict:
    """Take a line of text and parse it."""
    if not "ersetzt" in text:
        return None
    
    # parse the location
    location = []
    try:
        location.extend(re.search(r"§ \d{1,3}[a-z]? Absatz \d?", text).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Absatz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Satz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Nummer \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    
    # parse text to replace
    replace_text = [text[m.span()[0]:m.span()[1]] for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", text, re.MULTILINE)]
    
    return {"location": location, "text": replace_text, "how": "replace"}

In [555]:
def parse_rephrase(text: str) -> dict:
    """Take a line of text and parse it."""
    if not "gefasst" in text:
        return None
    
    # parse the location
    location = []
    try:
        location.extend(re.search(r"§ \d{1,3}[a-z]? Absatz \d?", text).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Absatz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Satz \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    try:
        location.extend(re.search(r"Nummer \d{1,3}", re.sub(r"(?<=„)(.|\n)*?(?=“)", "", text)).captures())
    except:
        pass
    
    # parse text to replace
    replace_text = [text[m.span()[0]:m.span()[1]] for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", text, re.MULTILINE)]
    
    return {"location": location, "text": replace_text, "how": "rephrase"}

In [556]:
changes = []
for change_request_line in clean_text.split("\n"):
    res_insert_after = parse_insert_after(change_request_line)
    res_replace = parse_replace(change_request_line)
    res_rephrase = parse_rephrase(change_request_line)
    print(res_insert_after, res_replace, res_rephrase)
    changes.append((res_insert_after, res_replace, res_rephrase))
    print()

{'location': ['§ 2', 'Absatz 2'], 'text': ['(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von vergleichbarer  Schwere  wie  die  Straf-  und Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere Verfehlunge n)  eingetragen,  insbesondere  vorsätzliche  oder  grob fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1 und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen Arbeitsorganisation. Der für die Eintragung erforderliche Nachweis der jeweiligen schweren Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige Behörde zu treffen.'], 'how': 'insert_after'} None None

None None None

{'location': ['Absatz 4', 'Satz 1

## Apply the changes to the source law

In [557]:
source_text = open("../data/0483-21_wettbewerbsgesetzt.txt","r").read()

In [558]:
# apply first change request
changes[0]

({'location': ['§ 2', 'Absatz 2'],
  'text': ['(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von vergleichbarer  Schwere  wie  die  Straf-  und Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere Verfehlunge n)  eingetragen,  insbesondere  vorsätzliche  oder  grob fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1 und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen Arbeitsorganisation. Der für die Eintragung erforderliche Nachweis der jeweiligen schweren Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige Behörde zu treffen.'],
  'how': 'insert_after'},
 None,
 None)

In [594]:
def apply_change(change_spec, source_text) -> str:
    # find line to apply the change to
    start_idx = 0
    for loc in change_spec["location"]:
        for idx, line in enumerate(source_text.split("\n")[start_idx:]):
            if loc.startswith("Absatz "):
                loc = loc.replace("Absatz ", "(") + ")"
            if line.startswith(loc):
                start_idx = idx
                break
    res_text = source_text.split("\n")
    res_text.insert(start_idx + 2, change_spec["text"][0])
    return "\n".join(res_text)

In [595]:
res = apply_change(changes[0][0], source_text)

print(res)

Gesetz zur Einrichtung und zum Betrieb eines Registers zum Schutz des Wettbewerbs um öffentliche Aufträge und Konzessionen (Wettbewerbsregistergesetz - WRegG)
Nichtamtliches Inhaltsverzeichnis

WRegG

Ausfertigungsdatum: 18.07.2017

Vollzitat:

"Wettbewerbsregistergesetz vom 18. Juli 2017 (BGBl. I S. 2739), das zuletzt durch Artikel 78 des Gesetzes vom 10. August 2021 (BGBl. I S. 3436) geändert worden ist"
Hinweis:	Geändert durch Art. 3 G v. 16.7.2021 I 2959
 	Änderung durch Art. 78 G v. 10.8.2021 I 3436 (Nr. 53) mWv 1.1.2024 noch nicht berücksichtigt

Näheres zur Standangabe finden Sie im Menü unter Hinweise
Fußnote

(+++ Textnachweis ab: 29.7.2017 +++)
(+++ Zur Anwendung vgl. § 5 Abs. 1 u. § 12 Abs. 2 +++)

 

Das G wurde als Artikel 1 des G v. 18.7.2017 I 2739 vom Bundestag beschlossen. Es ist gem. Art. 3 Abs. 1 dieses G am 29.7.2017 in Kraft getreten.
Nichtamtliches Inhaltsverzeichnis
§ 1 Einrichtung des Wettbewerbsregisters
(1) Beim Bundeskartellamt (Registerbehörde) wird ein Regi

## Apply it all to another document

In [514]:
# try a second document
filename2 = '../data/0145-21.pdf'

change_law_raw2 = read_pdf_law(filename2)

In [516]:
clean_text2 = preprocess(change_law_raw2)

In [521]:
print(clean_text2)

1. Die Inhaltsübersicht wird wie folgt geändert:a) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermäch-tigung“ angefügt.
1. Die Inhaltsübersicht wird wie folgt geändert:b) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:„§ 173 Zustellung elektronischer Dokumente§174 Zustellung durch Aushändigung an der Amtsstelle§175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis§176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.
2. § 130a wird wie folgt geändert:a) Der Überschrift werden ein Semikolon und das Wort „Verordnungsermächtigung“angefügt.
2. § 130a wird wie folgt geändert:b) Absatz 2 Satz 2 wird wie folgt gefasst:„Die Bundesregierung bestimmt durch Rechtsverordnung mit Zustimmung desBundesrates technische Rahmenbedingungen für die Übermittlung und Eignungzur Bearbeitung durch das Gericht.“
2. § 130a wird wie folgt geändert:c) Absatz 4 wird wie folgt geändert:a
2. § 130a wird wie folgt geändert:a) In Nummer 3 werden nach dem Wort „

In [517]:
for change_request_line in clean_text2.split("\n"):
    res_insert_after = parse_insert_after(change_request_line)
    res_replace = parse_replace(change_request_line)
    res_rephrase = parse_rephrase(change_request_line)
    print(res_insert_after, res_replace, res_rephrase)
    print()

None None None

None None {'location': [], 'text': ['§ 173 Zustellung elektronischer Dokumente§174 Zustellung durch Aushändigung an der Amtsstelle§175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis§176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag'], 'how': 'rephrase'}

None None None

None None {'location': [], 'text': ['Die Bundesregierung bestimmt durch Rechtsverordnung mit Zustimmung desBundesrates technische Rahmenbedingungen für die Übermittlung und Eignungzur Bearbeitung durch das Gericht.'], 'how': 'rephrase'}

None None None

None None None

{'location': ['§ 130a'], 'text': [], 'how': 'insert_after'} None None

None None None

None None None

None None None

None None {'location': ['§ 173 Absatz 2'], 'text': ['§ 175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis (1) Ein Schriftstück kann den in § 173 Absatz 2 Genannten gegen Empfangsbe-kenntnis zugestellt werden. (2) Eine Zustellung gegen Empfangsbekenntnis kann auch durch Telekopie erfol-g